# Hikaru Game Analysis

In [ ]:
import chess.pgn as ch
import pandas as pd
import re

def extract_game_data(game, game_id=1):
    """
    Extracts metadata and move data from a single PGN game.

    Parameters:
        game (chess.pgn.Game): A parsed PGN game object.
        game_id (int): Optional game ID for indexing.

    Returns:
        metadata_df (pd.DataFrame): Game metadata, indexed by Game_ID.
        moves_df (pd.DataFrame): Move list with times, indexed by move_number.
    """
    # --- Metadata ---
    metadata_df = pd.DataFrame([dict(game.headers)])
    metadata_df["Game_ID"] = game_id
    metadata_df.set_index("Game_ID", inplace=True)

    # --- Moves ---
    move_data = []
    node = game
    move_num = 1
    white_move = None
    white_time = None
    black_move = None
    black_time = None

    clk_pattern = re.compile(r"\[%clk ([0-9:.\-]+)\]")

    while node.variations:
        next_node = node.variation(0)
        move_san = next_node.san()
        comment = next_node.comment

        clk_match = clk_pattern.search(comment)
        clk_time = clk_match.group(1) if clk_match else None

        if node.board().turn:
            # White move
            white_move = move_san
            white_time = clk_time
        else:
            # Black move
            black_move = move_san
            black_time = clk_time
            move_data.append({
                "move_number": move_num,
                "white_move": white_move,
                "white_time": white_time,
                "black_move": black_move,
                "black_time": black_time
            })
            move_num += 1

        node = next_node

    moves_df = pd.DataFrame(move_data)
    moves_df.set_index("move_number", inplace=True)

    return metadata_df, moves_df


def extract_all_games(pgn_path):
    """
    Reads all games from a PGN file and returns:
      - metadata_df: Combined metadata for all games
      - moves_dict: Dictionary of move DataFrames keyed by Game_ID
    """
    all_metadata = []
    moves_dict = {}

    with open(pgn_path, encoding="utf-8") as pgn:
        game_id = 1
        while True:
            game = ch.read_game(pgn)
            if game is None:
                break  # End of file

            metadata_df, moves_df = extract_game_data(game, game_id)
            all_metadata.append(metadata_df)
            moves_dict[game_id] = moves_df
            game_id += 1

    # Combine all metadata
    metadata_df = pd.concat(all_metadata)
    return metadata_df, moves_dict
